# Setup
Ho avuto problemi con il mio neo4j locale quindi nel dubbio ho usato quello su docker
```sh
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```
E poi connessione su [localhost:7474](localhost:7474)

In [ ]:
! pip install llama-index llama-index-graph-stores-neo4j llama-index-embeddings-openai

In [4]:
import nest_asyncio; nest_asyncio.apply()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "" # Set your OpenAI API key here

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [1]:
import os
import uuid

# Create a temporary folder to store in-memory files in, which is removed after use.
# Takes a dictionary of the form {filename: bytes} so it also works for non-streamlit applications.
# Note that this removes all file metadata.
class TempDir:
    def __init__(self, files):
        # This assumes Unix-like filesystem.
        # Consider using the tempfile module to make it cross-platform
        self.tmpdir = os.path.join("/tmp/upload/", str(uuid.uuid4()))
        self.files = files

    def __enter__(self):
        os.makedirs(self.tmpdir)
        for filename, file_bytes in self.files.items():
            file_path = os.path.join(self.tmpdir, filename)
            with open(file_path, 'wb') as f:
                f.write(file_bytes)
        return self.tmpdir

    def __exit__(self, exc_type, exc_value, exc_traceback):
        for filename in self.files.keys():
            file_path = os.path.join(self.tmpdir, filename)
            os.remove(file_path)
        os.rmdir(self.tmpdir)
        return False

In [ ]:
file_dict = {"test": """Cartella Clinica di Dimissione
Paziente: Mario Rossi
Data di Dimissione: 26 novembre 2024

Diagnosi:

Polmonite interstiziale bilaterale di origine virale.
Trattamento Somministrato:

Terapia antibiotica con Ceftriaxone 1g/die per via endovenosa per 7 giorni.
Terapia di supporto con ossigenoterapia a basso flusso.
Esami Diagnostici:

Radiografia del torace: infiltrati polmonari bilaterali di tipo reticolare.
Emogasanalisi: lieve ipossiemia (PaO2 65 mmHg).
PCR (Proteina C-Reattiva): 85 mg/L (ridotta a 25 mg/L dopo la terapia).
Prossimi Passi e Raccomandazioni:

Visite di controllo: appuntamento con il pneumologo tra 14 giorni.
Continuare terapia domiciliare con Amoxicillina 500 mg ogni 8 ore per 5 giorni.
Eseguire nuova radiografia del torace tra 1 mese per monitorare il recupero.
Medico Responsabile:

Dr. Anna Bianchi
Reparto di Pneumologia, Ospedale Santa Maria, Roma
Telefono: +39 06 1234567
""".encode()}
with TempDir(file_dict) as tempdir:
    documents = SimpleDirectoryReader(input_dir=tempdir).load_data()

In [ ]:
#documents = SimpleDirectoryReader("/Users/marcocalamo/Downloads/txt").load_data()

In [5]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

In [35]:
# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="hackaton",
    url="bolt://localhost:7687",
)

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_ec67c859 FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:`__Node__`)\n            REQUIRE n.id IS UNIQUE;'
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_907a464e FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:`__Entity__`)\n            REQUIRE n.id IS U

In [ ]:
! pip install deeponto

In [6]:
# parse ontology
from deeponto.onto import Ontology

Please enter the maximum memory located to JVM [8g]:

INFO:deeponto:8g maximum memory allocated to JVM.
INFO:deeponto:JVM started successfully.


In [ ]:
onto = Ontology("/Users/marcocalamo/multi_ont.owl", "elk")

In [7]:
#from llama_index.llms.openai import OpenAI

#response = OpenAI().complete("Paul Graham is ")
#print(response)

In [ ]:
#stream = OpenAI().stream_complete("Hi, write a short story")

#s=''
#for r in stream:
#    print(r.delta, end="")
#    s += r.delta
#print()
#print('---------')
#print(s)

In [ ]:
onto.owl_classes

In [ ]:
ent = []
for s,_ in onto.owl_classes.items():
    print(s.split('#')[1])
    ent.append(s.split('#')[1])

In [ ]:
rel = []
for s,_ in onto.owl_object_properties.items():
    print(s.split('#')[1])
    rel.append(s.split('#')[1])

In [ ]:
for s,_ in onto.owl_data_properties.items():
    print(s.split('#')[1])

In [8]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

from llama_index.llms.openai import OpenAI

In [ ]:
llm = OpenAI("gpt-4-turbo")


# recommended uppercase, underscore separated
entities = Literal[tuple([re.sub(r'(?<!^)(?=[A-Z])', '_', e).upper() for e in ent])]
relations = Literal[tuple([re.sub(r'(?<!^)(?=[A-Z])', '_', r).upper() for r in rel])]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    #kg_validation_schema=schema,
    strict=False,  # if false, will allow triplets outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

"""
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
import re

kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=[re.sub(r'(?<!^)(?=[A-Z])', '_', e).upper() for e in ent],
    allowed_relation_types=[re.sub(r'(?<!^)(?=[A-Z])', '_', r).upper() for r in rel],
)
"""

In [9]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.embeddings.openai import OpenAIEmbedding

In [ ]:
# create
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

# Format Everything

In [10]:
def compute_label_usage_percentage(uri, user, password, reference_labels):
    """
    Computes the percentage of distinct labels in the database that are part of a reference list.

    Args:
        uri (str): The URI of the Neo4j database (e.g., "bolt://localhost:7687").
        user (str): The username for authentication.
        password (str): The password for authentication.
        reference_labels (list): A list of reference labels to compare against.

    Returns:
        float: The percentage of distinct labels used in the database that match the reference list.
        list: The labels found in both the database and the reference list.
    """
    query = """
    MATCH (n)
    WITH DISTINCT labels(n) AS labels
    UNWIND labels AS label
    RETURN DISTINCT label
    """
    try:
        # Initialize the driver
        driver = GraphDatabase.driver(uri, auth=(user, password))
        
        # Run the query and collect results
        with driver.session() as session:
            result = session.run(query)
            db_labels = {record["label"] for record in result}  # Use a set for fast lookups
        
        # Find the intersection of database labels and reference labels
        reference_set = set(reference_labels)
        common_labels = db_labels.intersection(reference_set)
        
        # Calculate the percentage
        percentage = (len(common_labels) / len(reference_set)) * 100 if reference_set else 0
        
        return percentage, list(common_labels)
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0, []
    
    finally:
        # Close the driver
        driver.close()

In [11]:
def compute_relationship_type_usage_percentage(uri, user, password, reference_relationship_types):
    """
    Computes the percentage of distinct relationship types in the database that are part of a reference list.

    Args:
        uri (str): The URI of the Neo4j database (e.g., "bolt://localhost:7687").
        user (str): The username for authentication.
        password (str): The password for authentication.
        reference_relationship_types (list): A list of reference relationship types to compare against.

    Returns:
        float: The percentage of distinct relationship types used in the database that match the reference list.
        list: The relationship types found in both the database and the reference list.
    """
    query = """
    MATCH ()-[r]-()
    RETURN DISTINCT type(r) AS relationship_type
    """
    try:
        # Initialize the driver
        driver = GraphDatabase.driver(uri, auth=(user, password))
        
        # Run the query and collect results
        with driver.session() as session:
            result = session.run(query)
            db_relationship_types = {record["relationship_type"] for record in result}  # Use a set for fast lookups
        
        # Find the intersection of database relationship types and reference list
        reference_set = set(reference_relationship_types)
        common_relationship_types = db_relationship_types.intersection(reference_set)
        
        # Calculate the percentage
        percentage = (len(common_relationship_types) / len(reference_set)) * 100 if reference_set else 0
        
        return percentage, list(common_relationship_types)
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0, []
    
    finally:
        # Close the driver
        driver.close()


In [12]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "hackaton"

In [ ]:
compute_label_usage_percentage(uri, user, password, [re.sub(r'(?<!^)(?=[A-Z])', '_', e).upper() for e in ent])

In [ ]:
compute_relationship_type_usage_percentage(uri, user, password, [re.sub(r'(?<!^)(?=[A-Z])', '_', e).upper() for e in rel])

In [67]:
def docs2neo4j(documents):
    llm = OpenAI("gpt-4o-mini")

    ent, rel = extract_ent_rel_2()

    # recommended uppercase, underscore separated
    entities = Literal[tuple([re.sub(r'(?<!^)(?=[A-Z])', '_', e).upper() for e in ent])]
    relations = Literal[tuple([re.sub(r'(?<!^)(?=[A-Z])', '_', r).upper() for r in rel])]
    
    
    kg_extractor = SchemaLLMPathExtractor(
        llm=llm,
        possible_entities=entities,
        possible_relations=relations,
        #kg_validation_schema=schema,
        strict=False,  # if false, will allow triplets outside of the schema
        num_workers=4,
        max_triplets_per_chunk=10,
    )

    # Note: used to be `Neo4jPGStore`
    graph_store = Neo4jPropertyGraphStore(
        username="neo4j",
        password="hackaton",
        url="bolt://localhost:7687",
    )

    index = PropertyGraphIndex.from_documents(
        documents,
        kg_extractors=[kg_extractor],
        property_graph_store=graph_store,
        show_progress=True,
    )

    

In [16]:
def pialla_neo4j():
    query = """
        MATCH (n)
        DETACH DELETE (n)
        """
    try:
        # Initialize the driver
        driver = GraphDatabase.driver(uri, auth=(user, password))
        
        # Run the query and collect results
        with driver.session() as session:
            result = session.run(query)
    except Exception as e:
        print(f"An error occurred: {e}")
            

In [92]:
def txt2doc(txt):
    file_dict = {"test": open(txt, "r", errors='ignore').read().encode()}
    with TempDir(file_dict) as tempdir:
        documents = SimpleDirectoryReader(input_dir=tempdir).load_data()
    return documents

In [84]:
def extract_ent_rel():
    onto = Ontology("/Users/marcocalamo/multi_ont.owl", "elk")
    ent = ["Document", "Patient", "Treatment", "Medical Test","Diagnostic",
           "Result","Outcome","Therapy","Drug", "Family History Role",
           "Treatment Association", "Specialist", "Side Effect",
           "Comorbidity Interaction", "Test Dependency"]

    #for s,_ in onto.owl_classes.items():
    #    #print(s.split('#')[1])
    #    ent.append(s.split('#')[1])
        

    rel = ["hasPatient", "hasTreatment", "hasMedicalTest", "hasDiagnostic",
           "hasResult", "hasOutcome", "hasTherapy", "hasDrug", "hasFamilyHistoryRole",
           "hasTreatmentAssociation", "hasSpecialist", "hasSideEffect", "hasComorbidityInteraction", 
           "hasTestDependency"]
    #for s,_ in onto.owl_object_properties.items():
    #    #print(s.split('#')[1])
    #    rel.append(s.split('#')[1])

    entities = [re.sub(r'(?<!^)(?=[A-Z])', '_', e).upper() for e in ent]
    relations = [re.sub(r'(?<!^)(?=[A-Z])', '_', r).upper() for r in rel]

    #print(entities, relations)

    return entities, relations

def extract_ent_rel_2():
    onto = Ontology("/Users/marcocalamo/multi_ont.owl", "elk")
    ent = ["Document", "Patient", "Treatment", "Medical Test","Diagnostic",
       "Result","Outcome","Therapy","Drug", "Family History Role",
       "Treatment Association", "Specialist", "Side Effect",
       "Comorbidity Interaction", "Test Dependency"]

    #for s,_ in onto.owl_classes.items():
    #    #print(s.split('#')[1])
    #    ent.append(s.split('#')[1])
        

    rel = ["hasPatient", "hasTreatment", "hasMedicalTest", "hasDiagnostic",
           "hasResult", "hasOutcome", "hasTherapy", "hasDrug", "hasFamilyHistoryRole",
           "hasTreatmentAssociation", "hasSpecialist", "hasSideEffect", "hasComorbidityInteraction", 
           "hasTestDependency"]

    return ent, rel
    

In [97]:
import openpyxl
from openpyxl import load_workbook


def valuta_txt_ent():
    ent, rel = extract_ent_rel()
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "hackaton"
    res = compute_label_usage_percentage(uri, user, password, ent)

    return res

def valuta_txt_rel():
    ent, rel = extract_ent_rel()
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "hackaton"
    res = compute_relationship_type_usage_percentage(uri, user, password, rel)

    return res
    
    

def genera_excel_con_valutazione(input_txt, output_excel):
    """
    Legge un file txt, chiama una funzione di valutazione e salva i risultati in un file Excel.
    
    Args:
        input_txt (str): Percorso del file .txt da valutare.
        output_excel (str): Percorso del file .xlsx generato.
    """

    nome_txt = input_txt.split("/")[-1]  # Estrae il nome del file

    # Crea doc
    document = txt2doc(input_txt)
    
    # Riempi neo4j
    docs2neo4j(document)
    
    # Ottieni la valutazione e le entità trovate
    valutazione, entita_trovate = valuta_txt_ent()
    
    # Tronca la valutazione al secondo decimale
    valutazione_troncata = round(valutazione, 2)
    
    # Prepara i dati da salvare
    entita_str = ", ".join(entita_trovate)  # Formatta le entità come stringa

    # Ottieni la valutazione e le entità trovate
    valutazione_rel, rel_trovate = valuta_txt_rel()
    
    # Tronca la valutazione al secondo decimale
    valutazione_r_troncata = round(valutazione_rel, 2)
    
    # Prepara i dati da salvare
    rel_str = ", ".join(rel_trovate)  # Formatta le entità come stringa
    
   # Verifica se il file Excel esiste
    if os.path.exists(output_excel):
        # Carica il file esistente
        wb = load_workbook(output_excel)
        ws = wb.active
    else:
        # Crea un nuovo file Excel
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "Valutazioni"
        # Scrivi l'intestazione
        ws.append(["nome_txt", "valutazione_ent", "entita_trovate", "valutazione_rel", "rel_trovate"])
    
    # Scrivi i dati
    ws.append([nome_txt, valutazione_troncata, entita_str, valutazione_r_troncata, rel_str])
    
    # Salva il file Excel
    wb.save(output_excel)
    print(f"File Excel salvato in: {output_excel}")

    pialla_neo4j()


# Valuta txt

In [29]:
from neo4j import GraphDatabase
import re

In [88]:
pialla_neo4j()

In [99]:
# Esempio di utilizzo
# iterate through all file
path = "/Users/marcocalamo/Downloads/Clinical_History/"
for file in os.listdir(path): 
    # Check whether file is in text format or not 
    if file.endswith(".txt"): 
        file_path = f"{path}{file}"
  
        # call read text file function 
        genera_excel_con_valutazione(file_path, "valutazioni_clinical.xlsx")

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_ec67c859 FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:`__Node__`)\n            REQUIRE n.id IS UNIQUE;'
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_907a464e FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:`__Entity__`)\n            REQUIRE n.id IS U

Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████████████████| 1/1 [00:00<00:00,  1.09it/s]
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_ec67c859 FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:`__Node__`)\n            REQUIRE n.id IS UNIQUE;'
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_907a464e FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CR

File Excel salvato in: valutazioni_clinical_.xlsx


Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████████████████| 1/1 [00:01<00:00,  1.31s/it]
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_ec67c859 FOR (e:__Node__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:`__Node__`)\n            REQUIRE n.id IS UNIQUE;'
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_907a464e FOR (e:__Entity__) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CR

File Excel salvato in: valutazioni_clinical_.xlsx


Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting paths from text with schema:  40%|██   | 2/5 [00:07<00:09,  3.23s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Generating embeddings: 100%|██████████████████████| 1/1 [00:00<00:00,  3.82it/s]


File Excel salvato in: valutazioni_clinical_.xlsx
